# Deutsch's Algorithm: The First Quantum Speedup

## Introduction

Deutsch's algorithm, proposed by David Deutsch in 1985, was the first quantum algorithm to demonstrate a computational speedup over classical algorithms. While the problem it solves is relatively simple, it beautifully illustrates the key principles of quantum computing: superposition and quantum parallelism.

## The Problem

Given a black-box function (oracle) $f: \{0,1\} \rightarrow \{0,1\}$, determine whether the function is:
- **Constant**: $f(0) = f(1)$ (always outputs 0 or always outputs 1)
- **Balanced**: $f(0) \neq f(1)$ (outputs 0 for one input and 1 for the other)

### Classical Solution

Classically, you must query the function **twice** (once for $f(0)$ and once for $f(1)$) to determine if it's constant or balanced.

### Quantum Solution

Deutsch's algorithm solves this with **only one query** to the quantum oracle, demonstrating quantum parallelism.

## The Four Possible Functions

There are exactly four functions $f: \{0,1\} \rightarrow \{0,1\}$:

1. **Constant-0**: $f(0) = 0, f(1) = 0$ (always returns 0)
2. **Constant-1**: $f(0) = 1, f(1) = 1$ (always returns 1)
3. **Identity**: $f(0) = 0, f(1) = 1$ (balanced - returns the input)
4. **NOT**: $f(0) = 1, f(1) = 0$ (balanced - returns the negation of input)

## The Quantum Circuit

The Deutsch algorithm circuit follows this structure:

```
|0⟩ ─── H ─── [  Uf  ] ─── H ─── Measure
              [       ]
|1⟩ ─── H ─── [       ]
```

### Steps:

1. **Initialization**: Start with $|0\rangle$ and $|1\rangle$
2. **Hadamard gates**: Create superposition on both qubits
   - $|0\rangle \xrightarrow{H} \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$
   - $|1\rangle \xrightarrow{H} \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle)$
3. **Oracle $U_f$**: Apply the function as a quantum gate
4. **Final Hadamard**: Apply H to the first qubit
5. **Measurement**: Measure the first qubit
   - If **0**: function is constant
   - If **1**: function is balanced

### Why It Works

The key insight is the **phase kickback** mechanism. After the oracle:
- For constant functions: the phase is global, giving $\pm|+\rangle$
- For balanced functions: the phase is relative, giving $\pm|-\rangle$

The final Hadamard transforms:
- $|+\rangle \rightarrow |0\rangle$ (constant)
- $|-\rangle \rightarrow |1\rangle$ (balanced)

In [ ]:
# Import necessary libraries
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_bloch_multivector
import matplotlib.pyplot as plt

print("Qiskit imported successfully!")

## Oracle Implementations

Let's implement the four oracle functions as quantum circuits.

In [ ]:
def constant_0_oracle():
    """
    Constant function: f(x) = 0 for all x
    Oracle does nothing (identity operation)
    """
    oracle = QuantumCircuit(2, name='f(x)=0')
    # Do nothing - this is the identity
    return oracle

def constant_1_oracle():
    """
    Constant function: f(x) = 1 for all x
    Oracle flips the second qubit
    """
    oracle = QuantumCircuit(2, name='f(x)=1')
    oracle.x(1)  # Flip the output qubit
    return oracle

def identity_oracle():
    """
    Balanced function: f(x) = x (identity)
    Oracle performs CNOT with input as control
    """
    oracle = QuantumCircuit(2, name='f(x)=x')
    oracle.cx(0, 1)  # CNOT: if x=1, flip output
    return oracle

def not_oracle():
    """
    Balanced function: f(x) = NOT x
    Oracle performs X then CNOT
    """
    oracle = QuantumCircuit(2, name='f(x)=¬x')
    oracle.x(0)      # Flip input
    oracle.cx(0, 1)  # CNOT
    oracle.x(0)      # Flip input back
    return oracle

## Building the Deutsch Algorithm Circuit

In [ ]:
def deutsch_algorithm(oracle):
    """
    Implements Deutsch's algorithm with a given oracle.
    
    Args:
        oracle: A 2-qubit quantum circuit implementing the function f
    
    Returns:
        QuantumCircuit: The complete Deutsch algorithm circuit
    """
    # Create quantum circuit with 2 qubits and 1 classical bit
    qr = QuantumRegister(2, 'q')
    cr = ClassicalRegister(1, 'c')
    circuit = QuantumCircuit(qr, cr)
    
    # Step 1: Initialize second qubit to |1⟩
    circuit.x(1)
    circuit.barrier()
    
    # Step 2: Apply Hadamard gates to both qubits
    circuit.h(0)
    circuit.h(1)
    circuit.barrier()
    
    # Step 3: Apply the oracle
    circuit.compose(oracle, inplace=True)
    circuit.barrier()
    
    # Step 4: Apply Hadamard to first qubit
    circuit.h(0)
    circuit.barrier()
    
    # Step 5: Measure the first qubit
    circuit.measure(0, 0)
    
    return circuit

## Testing All Four Functions

Let's run Deutsch's algorithm on all four possible functions and visualize the results.

In [ ]:
# Create the four oracles
oracles = {
    'Constant f(x)=0': constant_0_oracle(),
    'Constant f(x)=1': constant_1_oracle(),
    'Balanced f(x)=x': identity_oracle(),
    'Balanced f(x)=¬x': not_oracle()
}

# Visualize each oracle circuit
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, (name, oracle) in enumerate(oracles.items()):
    oracle.draw('mpl', ax=axes[idx])
    axes[idx].set_title(f'Oracle: {name}')

plt.tight_layout()
plt.show()

In [ ]:
# Run Deutsch's algorithm on each oracle
simulator = AerSimulator()
results = {}

print("Running Deutsch's Algorithm on all oracles...\n")
print("=" * 60)

for name, oracle in oracles.items():
    # Build the circuit
    circuit = deutsch_algorithm(oracle)
    
    # Run the circuit
    job = simulator.run(circuit, shots=1000)
    result = job.result()
    counts = result.get_counts()
    
    results[name] = counts
    
    # Interpret the result
    measurement = list(counts.keys())[0]
    function_type = "CONSTANT" if measurement == '0' else "BALANCED"
    
    print(f"Oracle: {name}")
    print(f"  Measurement: {measurement}")
    print(f"  Result: {function_type}")
    print(f"  Counts: {counts}")
    print("-" * 60)

## Visualizing the Results

In [ ]:
# Plot histograms for all results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, (name, counts) in enumerate(results.items()):
    plot_histogram(counts, ax=axes[idx])
    axes[idx].set_title(f'{name}')
    axes[idx].set_ylim([0, 1100])

plt.tight_layout()
plt.show()

## Visualizing a Complete Circuit

Let's look at the complete Deutsch algorithm circuit for one example.

In [ ]:
# Show the complete circuit for the balanced identity function
example_circuit = deutsch_algorithm(identity_oracle())
print("Complete Deutsch Algorithm Circuit for f(x) = x (Balanced):")
example_circuit.draw('mpl', style='iqp')

## Understanding the Quantum Speedup

### Classical Complexity
- **Queries needed**: 2 (must evaluate both $f(0)$ and $f(1)$)
- **Deterministic**: Always requires 2 queries

### Quantum Complexity
- **Queries needed**: 1 (evaluates both inputs simultaneously via superposition)
- **Deterministic**: Always gives the correct answer in 1 query

### Why This is a Speedup

1. **Quantum Parallelism**: The first qubit is in superposition $\frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$, so the oracle is applied to both $|0\rangle$ and $|1\rangle$ simultaneously in a single query.

2. **Phase Kickback**: The second qubit (initialized to $|-\rangle$) encodes the function output as a phase:
   - When $f(x) = 0$: no phase change
   - When $f(x) = 1$: adds a phase of $-1$

3. **Interference**: The final Hadamard gate causes constructive or destructive interference:
   - Constant functions → constructive interference → measure 0
   - Balanced functions → destructive interference → measure 1

### Practical Demonstration

Let's verify that we only need one oracle call by examining the circuit depth.

In [ ]:
print("Query Comparison:\n")
print("Classical Algorithm:")
print("  - Must call f(0) and f(1) separately")
print("  - Total queries: 2")
print("  - Can use randomness but worst case still needs 2 queries\n")

print("Deutsch's Quantum Algorithm:")
for name, oracle in oracles.items():
    circuit = deutsch_algorithm(oracle)
    # Count oracle applications (excluding initialization, Hadamards, and measurement)
    print(f"  {name}:")
    print(f"    - Oracle applications: 1")
    print(f"    - Circuit depth: {circuit.depth()}")
    print(f"    - Result: Always correct with 100% probability")

## Mathematical Walkthrough

Let's trace through the quantum state at each step for a balanced function (e.g., $f(x) = x$):

1. **Initial state**: $|\psi_0\rangle = |0\rangle|1\rangle$

2. **After Hadamards**: 
   $$|\psi_1\rangle = \frac{1}{2}(|0\rangle + |1\rangle)(|0\rangle - |1\rangle)$$

3. **After Oracle** (for $f(x) = x$, which is a CNOT):
   $$|\psi_2\rangle = \frac{1}{2}(|0\rangle(|0\rangle - |1\rangle) + |1\rangle(|1\rangle - |0\rangle))$$
   $$= \frac{1}{2}(|0\rangle - |1\rangle)(|0\rangle - |1\rangle)$$
   $$= \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle) \otimes \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle)$$
   $$= |-\rangle \otimes |-\rangle$$

4. **After final Hadamard on first qubit**:
   $$|\psi_3\rangle = H|-\rangle \otimes |-\rangle = |1\rangle \otimes |-\rangle$$

5. **Measurement**: First qubit is $|1\rangle$ → **Balanced**

For a constant function, the phase information would be global, leading to $|0\rangle$ after the final Hadamard.

## Conclusion

Deutsch's algorithm demonstrates:

1. **Quantum Parallelism**: Evaluating multiple inputs simultaneously through superposition
2. **Phase Kickback**: Encoding classical information in quantum phases
3. **Quantum Interference**: Using destructive/constructive interference to extract global properties
4. **Provable Speedup**: Reducing query complexity from 2 to 1 (a factor of 2 improvement)

While the problem itself is not practically useful, Deutsch's algorithm:
- Was the first to show quantum computers can outperform classical computers
- Introduced techniques used in more powerful algorithms (like Deutsch-Jozsa, Simon's, and Shor's algorithms)
- Demonstrated that quantum speedup comes from exploiting quantum interference, not just parallelism

This simple 2-qubit circuit laid the foundation for quantum algorithms that would follow!